In [ ]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import tensorflow as tf
import time
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.layers import SimpleRNN, LSTM, Dense, Dropout, Masking, Embedding, Dense, Activation, Flatten, LSTM, TimeDistributed, RepeatVector,Input

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
ds_train = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/train.csv')

In [ ]:
ds_asset = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/asset_details.csv')
ds_example = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/example_test.csv')
ds_suplement_train = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/supplemental_train.csv')
ds_example_submission = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/example_sample_submission.csv')

In [ ]:
ds_train.head()

In [ ]:
ds_asset.head(100).sort_values(by=['Asset_ID'])

In [ ]:
ds_example.head()
ds_example[ds_example['Asset_ID']==3]

In [ ]:
ds_suplement_train.head()

In [ ]:
ds_example_submission.head()

In [ ]:
ds_train[ds_train['timestamp']==1514764860]

In [ ]:
ds_train[ds_train['Target']==np.nan]
ds_train.isna().sum()

In [ ]:
# Define plot space
fig, ax = plt.subplots(figsize=(10, 6))

# Create bar plot
ax.bar(ds_asset['Asset_Name'], 
       ds_asset['Weight'])
plt.xlabel('Asset Name')
plt.ylabel('Weight')
plt.xticks(rotation=70)
plt.show()

In [ ]:
ds_train_copy = ds_train
ds_train_copy['date'] = pd.to_datetime(ds_train_copy['timestamp'], unit='s')
ds_train_copy.head()

In [ ]:
eth = ds_train_copy[ds_train_copy['Asset_ID']==6].set_index('timestamp')
eth.head(100)

In [ ]:
eth.info(show_counts=True)

In [ ]:
eth.isna().sum()

In [ ]:
btcsh = ds_train_copy[ds_train_copy['Asset_ID']==2].set_index('timestamp')
bnc = ds_train_copy[ds_train_copy['Asset_ID']==0].set_index('timestamp')
btc = ds_train_copy[ds_train_copy['Asset_ID']==1].set_index('timestamp')
eos = ds_train_copy[ds_train_copy['Asset_ID']==5].set_index('timestamp')
eth_csc = ds_train_copy[ds_train_copy['Asset_ID']==7].set_index('timestamp')
ltc = ds_train_copy[ds_train_copy['Asset_ID']==9].set_index('timestamp')
mon = ds_train_copy[ds_train_copy['Asset_ID']==11].set_index('timestamp')
tro = ds_train_copy[ds_train_copy['Asset_ID']==13].set_index('timestamp')
ste = ds_train_copy[ds_train_copy['Asset_ID']==12].set_index('timestamp')
car = ds_train_copy[ds_train_copy['Asset_ID']==3].set_index('timestamp')
iot = ds_train_copy[ds_train_copy['Asset_ID']==8].set_index('timestamp')
mak = ds_train_copy[ds_train_copy['Asset_ID']==10].set_index('timestamp')
dog = ds_train_copy[ds_train_copy['Asset_ID']==4].set_index('timestamp')

In [ ]:
dog.head().count()

In [ ]:
btcsh.head()

In [ ]:
beg_btcsh = btcsh.index[0].astype('datetime64[s]')
end_btcsh = btcsh.index[-1].astype('datetime64[s]')
beg_bnc = bnc.index[0].astype('datetime64[s]')
end_bnc = bnc.index[-1].astype('datetime64[s]')
beg_btc = btc.index[0].astype('datetime64[s]')
end_btc = btc.index[-1].astype('datetime64[s]')
beg_eos = eos.index[0].astype('datetime64[s]')
end_eos = eos.index[-1].astype('datetime64[s]')
beg_eth_csc = eth_csc.index[0].astype('datetime64[s]')
end_eth_csc = eth_csc.index[-1].astype('datetime64[s]')
beg_ltc = ltc.index[0].astype('datetime64[s]')
end_ltc = ltc.index[-1].astype('datetime64[s]')
beg_mon = mon.index[0].astype('datetime64[s]')
end_mon = mon.index[-1].astype('datetime64[s]')
beg_tro = tro.index[0].astype('datetime64[s]')
end_tro = tro.index[-1].astype('datetime64[s]')
beg_eth = eth.index[0].astype('datetime64[s]')
end_eth = eth.index[-1].astype('datetime64[s]')
beg_ste = ste.index[0].astype('datetime64[s]')
end_ste = ste.index[-1].astype('datetime64[s]')
beg_car = car.index[0].astype('datetime64[s]')
end_car = car.index[-1].astype('datetime64[s]')
beg_iot = iot.index[0].astype('datetime64[s]')
end_iot = iot.index[-1].astype('datetime64[s]')
beg_mak = mak.index[0].astype('datetime64[s]')
end_mak = mak.index[-1].astype('datetime64[s]')
# beg_dog = dog.index[0].astype('datetime64[s]')
# end_dog = dog.index[-1].astype('datetime64[s]')
print('Bitcoin Cash     :', beg_btcsh, 'to', end_btcsh)
print('Binance Coin     :', beg_bnc, 'to', end_bnc)
print('Bitcoin          :', beg_btc, 'to', end_btc)
print('EOS IO           :', beg_eos, 'to', end_eos)
print('Etherium Classic :', beg_eth_csc, 'to', end_eth_csc)
print('Ethereum         :', beg_eth, 'to', end_eth)
print('Lite Coin        :', beg_ltc, 'to', end_ltc)
print('Monero           :', beg_mon, 'to', end_mon)
print('TRON             :', beg_tro, 'to', end_tro)
print('Stellar          :', beg_ste, 'to', end_ste)
print('Cardano          :', beg_car, 'to', end_car)
print('IOTA             :', beg_iot, 'to', end_iot)
print('Maker            :', beg_mak, 'to', end_mak)
# print('Dogecoin         :', beg_dog, 'to', end_dog)

In [ ]:
(eth.index[1:]-eth.index[:-1]).value_counts().head()

In [ ]:
eth = eth.reindex(range(eth.index[0],eth.index[-1]+60,60),method='pad')

In [ ]:
(eth.index[1:]-eth.index[:-1]).value_counts().head()

In [ ]:
(btc.index[1:]-btc.index[:-1]).value_counts().head()

In [ ]:
btc = btc.reindex(range(btc.index[0],btc.index[-1]+60,60),method='pad')
(btc.index[1:]-btc.index[:-1]).value_counts().head()

In [ ]:
(btcsh.index[1:]-btcsh.index[:-1]).value_counts().head()

In [ ]:
btcsh = btcsh.reindex(range(btcsh.index[0],btcsh.index[-1]+60,60),method='pad')
(btcsh.index[1:]-btcsh.index[:-1]).value_counts().head()

In [ ]:
(bnc.index[1:]-bnc.index[:-1]).value_counts().head()

In [ ]:
bnc = bnc.reindex(range(bnc.index[0],bnc.index[-1]+60,60),method='pad')
(bnc.index[1:]-bnc.index[:-1]).value_counts().head()

In [ ]:
(eos.index[1:]-eos.index[:-1]).value_counts().head()

In [ ]:
eos = eos.reindex(range(eos.index[0],eos.index[-1]+60,60),method='pad')
(eos.index[1:]-eos.index[:-1]).value_counts().head()

In [ ]:
(eth_csc.index[1:]-eth_csc.index[:-1]).value_counts().head()

In [ ]:
eth_csc = eth_csc.reindex(range(eth_csc.index[0],eth_csc.index[-1]+60,60),method='pad')
(eth_csc.index[1:]-eth_csc.index[:-1]).value_counts().head()

In [ ]:
(ltc.index[1:]-ltc.index[:-1]).value_counts().head()

In [ ]:
ltc = ltc.reindex(range(ltc.index[0],ltc.index[-1]+60,60),method='pad')
(ltc.index[1:]-ltc.index[:-1]).value_counts().head()

In [ ]:
(mon.index[1:]-mon.index[:-1]).value_counts().head()

In [ ]:
mon = mon.reindex(range(mon.index[0],mon.index[-1]+60,60),method='pad')
(mon.index[1:]-mon.index[:-1]).value_counts().head()

In [ ]:
(tro.index[1:]-tro.index[:-1]).value_counts().head()

In [ ]:
tro = tro.reindex(range(tro.index[0],tro.index[-1]+60,60),method='pad')
(tro.index[1:]-tro.index[:-1]).value_counts().head()

In [ ]:
(ste.index[1:]-ste.index[:-1]).value_counts().head()

In [ ]:
ste = ste.reindex(range(ste.index[0],ste.index[-1]+60,60),method='pad')
(ste.index[1:]-ste.index[:-1]).value_counts().head()

In [ ]:
(car.index[1:]-car.index[:-1]).value_counts().head()

In [ ]:
car = car.reindex(range(car.index[0],car.index[-1]+60,60),method='pad')
(car.index[1:]-car.index[:-1]).value_counts().head()

In [ ]:
(iot.index[1:]-iot.index[:-1]).value_counts().head()

In [ ]:
iot = iot.reindex(range(iot.index[0],iot.index[-1]+60,60),method='pad')
(iot.index[1:]-iot.index[:-1]).value_counts().head()

In [ ]:
(mak.index[1:]-mak.index[:-1]).value_counts().head()

In [ ]:
mak = mak.reindex(range(mak.index[0],mak.index[-1]+60,60),method='pad')
(mak.index[1:]-mak.index[:-1]).value_counts().head()

In [ ]:
# Define plot space
fig, ax = plt.subplots(5, 3, figsize=(18, 22))

# Bitcoin Cash
ax[0, 0].plot(btcsh['Close'], label='BTCSH')
ax[0, 0].set_title('Bitcoin Cash')
ax[0, 1].plot(bnc['Close'], label='BNC')
ax[0, 1].set_title('Binance Coin')
ax[0, 2].plot(btc['Close'], label='BTC')
ax[0, 2].set_title('Bitcoin')
ax[1, 0].plot(eos['Close'], label='EOS')
ax[1, 0].set_title('EOS.IO')
ax[1, 1].plot(eth_csc['Close'], label='ETH_CSC')
ax[1, 1].set_title('Etherium Cash')
ax[1, 2].plot(ltc['Close'], label='LTC')
ax[1, 2].set_title('Lite Coin')
ax[2, 0].plot(mon['Close'], label='MON')
ax[2, 0].set_title('Monero')
ax[2, 1].plot(tro['Close'], label='TRO')
ax[2, 1].set_title('TRON')
ax[2, 2].plot(eth['Close'], label='ETH')
ax[2, 2].set_title('Etherium')
ax[3, 0].plot(ste['Close'], label='STE')
ax[3, 0].set_title('Stelar')
ax[3, 1].plot(car['Close'], label='CAR')
ax[3, 1].set_title('Cardano')
ax[3, 2].plot(iot['Close'], label='IOT')
ax[3, 2].set_title('IOTA')
ax[4, 0].plot(mak['Close'], label='MAK')
ax[4, 0].set_title('Maker')
plt.show()

In [ ]:
# define function to compute log returns
def log_return(series, periods=1):
    return np.log(series).diff(periods=periods)

In [ ]:
# auxiliary function, from datetime to timestamp
totimestamp = lambda s: np.int32(time.mktime(datetime.strptime(s, "%d/%m/%Y").timetuple()))

all_assets_2021 = pd.DataFrame([])
for asset_id, asset_name in zip(ds_asset.Asset_ID, ds_asset.Asset_Name):
    asset = ds_train[ds_train["Asset_ID"]==asset_id].set_index("timestamp")
    asset = asset.loc[totimestamp('01/01/2021'):totimestamp('01/05/2021')]
    asset = asset.reindex(range(asset.index[0],asset.index[-1]+60,60),method='pad')
    lret = log_return(asset.Close.fillna(0))[1:]
    all_assets_2021 = all_assets_2021.join(lret, rsuffix=asset_name, how="outer")

In [ ]:
plt.imshow(all_assets_2021.corr());
plt.yticks(ds_asset.Asset_ID.values, ds_asset.Asset_Name.values);
plt.xticks(ds_asset.Asset_ID.values, ds_asset.Asset_Name.values, rotation='vertical');
plt.colorbar();

In [ ]:
ds_asset.head()

In [ ]:
frames = [btcsh, bnc, btc, eos, eth_csc, eth, ltc, mon, tro, ste, car, iot, mak, dog]
final_dataset = pd.concat(frames)
final_dataset_sorted = final_dataset.sort_index()
final_dataset_sorted.head()

In [ ]:
final_dataset_sorted[final_dataset_sorted['Asset_ID']==6].head()

In [ ]:
final_dataset_sorted.join(ds_asset['Weight'], on='Asset_ID')

In [ ]:
final_dataset_2021 = final_dataset_sorted[final_dataset_sorted['date']>'2021-01-01 00:00:00']
final_dataset_2021.head()

In [ ]:
final_dataset_2021.isna().sum()

In [ ]:
final_dataset_2021['Target'] = final_dataset_2021['Target'].ffill()
final_dataset_2021['Target'] = final_dataset_2021['Target'].bfill()
final_dataset_2021.isna().sum()

In [ ]:
# Bitcoin Cash
ds_btcsh = final_dataset_2021[final_dataset_2021['Asset_ID']==2]
ds_btcsh = ds_btcsh[ds_btcsh['date']>='2021-07-01 00:01:00']
ds_btcsh = ds_btcsh[ds_btcsh['date']<='2021-08-30 23:59:59']
# Binance Coin
ds_bnc = final_dataset_2021[final_dataset_2021['Asset_ID']==0]
ds_bnc = ds_bnc[ds_bnc['date']>='2021-07-01 00:01:00']
ds_bnc = ds_bnc[ds_bnc['date']<='2021-08-30 23:59:59']
# Bitcoin
ds_btc = final_dataset_2021[final_dataset_2021['Asset_ID']==1]
ds_btc = ds_btc[ds_btc['date']>='2021-07-01 00:01:00']
ds_btc = ds_btc[ds_btc['date']<='2021-08-30 23:59:59']
# EOS
ds_eos = final_dataset_2021[final_dataset_2021['Asset_ID']==5]
ds_eos = ds_eos[ds_eos['date']>='2021-07-01 00:01:00']
ds_eos = ds_eos[ds_eos['date']<='2021-08-30 23:59:59']
# Etherium Classic
ds_eth_csc = final_dataset_2021[final_dataset_2021['Asset_ID']==7]
ds_eth_csc = ds_eth_csc[ds_eth_csc['date']>='2021-07-01 00:01:00']
ds_eth_csc = ds_eth_csc[ds_eth_csc['date']<='2021-08-30 23:59:59']
# Litecoin
ds_ltc = final_dataset_2021[final_dataset_2021['Asset_ID']==9]
ds_ltc = ds_ltc[ds_ltc['date']>='2021-07-01 00:01:00']
ds_ltc = ds_ltc[ds_ltc['date']<='2021-08-30 23:59:59']
# Monero
ds_mon = final_dataset_2021[final_dataset_2021['Asset_ID']==11]
ds_mon = ds_mon[ds_mon['date']>='2021-07-01 00:01:00']
ds_mon = ds_mon[ds_mon['date']<='2021-08-30 23:59:59']
# TRON
ds_tro = final_dataset_2021[final_dataset_2021['Asset_ID']==13]
ds_tro = ds_tro[ds_tro['date']>='2021-07-01 00:01:00']
ds_tro = ds_tro[ds_tro['date']<='2021-08-30 23:59:59']
# Stellar
ds_ste = final_dataset_2021[final_dataset_2021['Asset_ID']==12]
ds_ste = ds_ste[ds_ste['date']>='2021-07-01 00:01:00']
ds_ste = ds_ste[ds_ste['date']<='2021-08-30 23:59:59']
# Cardano
ds_car = final_dataset_2021[final_dataset_2021['Asset_ID']==3]
ds_car = ds_car[ds_car['date']>='2021-07-01 00:01:00']
ds_car = ds_car[ds_car['date']<='2021-08-30 23:59:59']
# IOTA
ds_iot = final_dataset_2021[final_dataset_2021['Asset_ID']==8]
ds_iot = ds_iot[ds_iot['date']>='2021-07-01 00:01:00']
ds_iot = ds_iot[ds_iot['date']<='2021-08-30 23:59:59']
# Maker 
ds_mak = final_dataset_2021[final_dataset_2021['Asset_ID']==10]
ds_mak = ds_mak[ds_mak['date']>='2021-07-01 00:01:00']
ds_mak = ds_mak[ds_mak['date']<='2021-08-30 23:59:59']
# Dogecoin 
ds_dog = final_dataset_2021[final_dataset_2021['Asset_ID']==4]
ds_dog = ds_dog[ds_dog['date']>='2021-07-01 00:01:00']
ds_dog = ds_dog[ds_dog['date']<='2021-08-30 23:59:59']
ds_dog.head()
# Etherium
ds_eth = final_dataset_2021[final_dataset_2021['Asset_ID']==6]
ds_eth = ds_eth[ds_eth['date']>='2021-07-01 00:01:00']
ds_eth = ds_eth[ds_eth['date']<='2021-08-30 23:59:59']
ds_eth.head()

In [ ]:
list_asset = [['Bitcoin Cash',ds_btcsh], 
              ['Binance Coin',ds_bnc], 
              ['Bitcoin',ds_btc], 
              ['EOS.IO',ds_eos], 
              ['Etherium Classic',ds_eth_csc], 
              ['Lite Coin',ds_ltc], 
              ['Monero',ds_mon], 
              ['TRON',ds_tro], 
              ['Stellar',ds_ste], 
              ['Cardano',ds_car], 
              ['IOT',ds_iot], 
              ['Maker',ds_mak], 
              ['Dogecoin',ds_dog],
              ['Etherium',ds_eth]]

In [ ]:
list_asset[13][1].head()

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
def x_normalization(dataset):
    data = dataset[['Target']].values
    sc   = MinMaxScaler()
    data_sc = sc.fit(data)
    x = data_sc.transform(data)
    return data

In [ ]:
def split_dataset(dataset):
    X_samples = list()
    y_samples = list()
    NumberOfRows = len(dataset)
    TimeSteps    = 15  
   
    for i in range(TimeSteps , NumberOfRows , 1):
        x_sample = dataset[i-TimeSteps:i]
        y_sample = dataset[i]
        X_samples.append(x_sample)
        y_samples.append(y_sample)

    X_data=np.array(X_samples)
    X_data=X_data.reshape(X_data.shape[0], X_data.shape[1], 1)
    
    y_data=np.array(y_samples)
    y_data=y_data.reshape(y_data.shape[0], 1)
    
    TestingRecords=500

    X_train=X_data[:-TestingRecords]
    X_test=X_data[-TestingRecords:]
    y_train=y_data[:-TestingRecords]
    y_test=y_data[-TestingRecords:]
    
    return X_train, y_train, X_test, y_test

In [ ]:
def get_model_ltsm(x_train):
    TimeSteps=x_train.shape[1]
    TotalFeatures=x_train.shape[2]
    model = Sequential()
    model.add(LSTM(units = 15, activation = 'relu', input_shape = (TimeSteps, TotalFeatures), return_sequences=True))
    model.add(LSTM(units = 10, activation = 'relu', input_shape = (TimeSteps, TotalFeatures), return_sequences=True))
    model.add(LSTM(units = 5, activation = 'relu', return_sequences=False ))
    model.add(Dropout(0.3))
    model.add(Dense(units = 1))
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')
    return model

In [ ]:
import time
for dataset in list_asset:
    asset_name = dataset[0]
    data       = x_normalization(dataset[1])
    x_train, y_train, x_test, y_test = split_dataset(data)
    model      = get_model_ltsm(x_train)
    model.summary()
    checkpoint = ModelCheckpoint(asset_name+'.h5', verbose=1, monitor='loss', save_best_only=True, mode='auto')
    StartTime  = time.time()
    print('Train for ' + asset_name)
    model.fit(x_train, 
              y_train, 
              batch_size = 15, 
              epochs = 3, 
              callbacks = [checkpoint])
    EndTime = time.time()
    print("## Total " + asset_name + " Time Taken: ", round((EndTime-StartTime)/60), 'Minutes ##')

In [ ]:
#Making predictions on test data
model.load_weights('./'+asset_name+'.h5')
predicted_Price = model.predict(x_test)
#predicted_Price = DataScaler.inverse_transform(predicted_Price)
print('#### Predicted Prices ####')
print(predicted_Price)
 
# Getting the original price values for testing data
orig=y_test
#orig=DataScaler.inverse_transform(y_test)
print('\n#### Original Prices ####')
print(orig)

In [ ]:
import gresearch_crypto
env = gresearch_crypto.make_env()
iter_test = env.iter_test()

trainDF = pd.read_csv('/kaggle/input/g-research-crypto-forecasting/train.csv', 
                 usecols=['Target', 'Asset_ID','timestamp'], dtype={'Asset_ID': 'int8'})
trainDF['datetime'] = pd.to_datetime(trainDF['timestamp'], unit='s')
trainDF = trainDF.set_index('datetime').drop('timestamp', axis=1)
trainDF = trainDF[(trainDF.index.year == 2021) & (trainDF.index.month > 5)]
dfs = {asset_id: trainDF[trainDF['Asset_ID'] == asset_id].resample('1min').interpolate().copy() for asset_id in trainDF['Asset_ID'].unique()}
del trainDF
for df_test, df_pred in iter_test:
    df_test['datetime'] = pd.to_datetime(df_test['timestamp'], unit='s')
    for _, row in df_test.iterrows():
        try:
            df = dfs[row['Asset_ID']]
            predicted = df.predict(pd.DataFrame([x_test]))[0]
            df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = predicted
        except:
            df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = 0
    df_pred['Target'] = df_pred['Target'].fillna(0)
    env.predict(df_pred)